In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import efficientnet.keras as efn
import matplotlib.pyplot as plt
import itertools
import os
import shutil
import random
from sklearn.metrics import confusion_matrix
import datetime
from numpy.random import seed
seed(1)

In [2]:
#Second way to load the data
#If we want the data to be broken up into Train, validation and Testing
# Change Benign and Malignant to 0 and 1 resp
os.chdir('/Users/nalisha/Documents/Winter/DepLearning/Project/images')
if os.path.isdir('train/0/') is False:
    os.mkdir('train')
    os.mkdir('valid')
    os.mkdir('test')
    random.seed(2021)
    
    for i in range(0, 2):
        shutil.move(f'{i}', 'train')
        os.mkdir(f'valid/{i}')
        os.mkdir(f'test/{i}')
        
        valid_samples = random.sample(os.listdir(f'train/{i}'), 788)
        for j in valid_samples:
            shutil.move(f'train/{i}/{j}', f'valid/{i}')
            
        test_samples = random.sample(os.listdir(f'train/{i}'), 394)
        for k in test_samples:
            shutil.move(f'train/{i}/{k}', f'test/{i}')
            
os.chdir('../..')

In [3]:
model_o = efn.EfficientNetB0()

In [4]:
# train/valid/test - 70/20/10
train_path = '/Users/nalisha/Documents/Winter/DepLearning/Project/images/train'
valid_path = '/Users/nalisha/Documents/Winter/DepLearning/Project/images/valid'
test_path = '/Users/nalisha/Documents/Winter/DepLearning/Project/images/test'

train_batch = ImageDataGenerator(preprocessing_function = efn.preprocess_input).flow_from_directory(
    directory = train_path, target_size = (224,224), batch_size = 10)
valid_batch = ImageDataGenerator(preprocessing_function = efn.preprocess_input).flow_from_directory(
    directory = valid_path, target_size = (224,224), batch_size = 10)
test_batch = ImageDataGenerator(preprocessing_function = efn.preprocess_input).flow_from_directory(
    directory = test_path, target_size = (224,224), batch_size = 10, shuffle = False)

Found 5545 images belonging to 2 classes.
Found 1576 images belonging to 2 classes.
Found 788 images belonging to 2 classes.


In [5]:
model_o.summary()

Model: "efficientnet-b0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 112, 112, 32) 0           stem_bn[0][0]                    
____________________________________________________________________________________

In [6]:
# input the pretrained model from keras without the FC layers 
x = model_o.layers[-1].output # discarding the last 1 layers
output = Dense(units = 2, activation = 'softmax')(x) # Adding the dense layer and softmax activation
model = Model(inputs = model_o.input, outputs=output) # Making a new model
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 112, 112, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 112, 112, 32) 0           stem_bn[0][0]                    
_______________________________________________________________________________________

In [7]:
#Freezing all layers except the last 2 layers
for layer in model.layers[:-2]:
    layer.trainable = False
    
#model.summary()

In [8]:
model.compile(optimizer = Adam(lr=0.0001), loss ='categorical_crossentropy', metrics = ['accuracy'])

In [18]:

path = "/Users/nalisha/Documents/Winter/DepLearning/Project/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorBoard = tf.keras.callbacks.TensorBoard(log_dir=path, histogram_freq=0, write_graph=True,
                                                          write_grads=False, write_images=False, embeddings_freq=0,
                                                          embeddings_layer_names=None, embeddings_metadata=None,
                                                          embeddings_data=None, update_freq=50)

CheckPoint = tf.keras.callbacks.ModelCheckpoint(filepath='/tmp/weights.hdf5', monitor='val_loss', verbose=0,
                                                               save_best_only=False, save_weights_only=False, mode='auto',
                                                               save_freq=2000)
callbacks = [CheckPoint, tensorBoard]

In [19]:
model.fit(x=train_batch, validation_data = valid_batch, epochs = 20 , verbose = 2, callbacks = callbacks)

Epoch 1/20
555/555 - 199s - loss: 0.4623 - accuracy: 0.8220 - val_loss: 0.6007 - val_accuracy: 0.6701
Epoch 2/20
555/555 - 195s - loss: 0.4379 - accuracy: 0.8399 - val_loss: 0.5868 - val_accuracy: 0.7132
Epoch 3/20
555/555 - 195s - loss: 0.4182 - accuracy: 0.8590 - val_loss: 0.5725 - val_accuracy: 0.7418
Epoch 4/20
555/555 - 194s - loss: 0.3970 - accuracy: 0.8756 - val_loss: 0.5597 - val_accuracy: 0.7640
Epoch 5/20
555/555 - 193s - loss: 0.3833 - accuracy: 0.8799 - val_loss: 0.5447 - val_accuracy: 0.7824
Epoch 6/20
555/555 - 197s - loss: 0.3716 - accuracy: 0.8828 - val_loss: 0.5298 - val_accuracy: 0.7976
Epoch 7/20
555/555 - 195s - loss: 0.3580 - accuracy: 0.8936 - val_loss: 0.5269 - val_accuracy: 0.7989
Epoch 8/20
555/555 - 195s - loss: 0.3481 - accuracy: 0.8905 - val_loss: 0.5149 - val_accuracy: 0.8077
Epoch 9/20
555/555 - 202s - loss: 0.3370 - accuracy: 0.8918 - val_loss: 0.4970 - val_accuracy: 0.8160
Epoch 10/20
555/555 - 200s - loss: 0.3316 - accuracy: 0.8941 - val_loss: 0.4869 - 

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir /Users/nalisha/Documents/Winter/DepLearning/Project/logs/fit/